<a href="https://colab.research.google.com/github/shinji-kondou/shinji-kondou.github.io/blob/main/20260130_Kakusei_advanced.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 75.6 MB/s eta 0:00:00


ChemBERTa: seyonec/ChemBERTa-zinc-base-v1(Updated May 20, 2021)

In [ ]:
# ============================================
# SMILES(分子丸ごと) → 事前学習モデル埋め込み → 学習/外挿評価
# - CEがNaNの行はすべて除外
# - SMILESが壊れている行も除外（RDKitで検証）
# - カチオン/アニオンは別々に埋め込み→結合（おすすめ）
# - 「未知カチオン」「未知アニオン」「未知イオンペア」で外挿評価（LeaveOneGroupOut）
# ============================================

import os
import re
import numpy as np
import pandas as pd

# ---- 入力ファイル ----
CSV_PATH = "/content/cation_anion_26kept_withChelpG_fragments_DFT_extended_catprefix_15features_preprocessed.csv"

# ---- 列名（あなたのCSVに合わせる） ----
COL_CAT = "SMILES_cation"
COL_ANI = "SMILES_anion"
COL_CE  = "CE"          # CE（数値 or 文字列）を想定

# ---- 事前学習モデル（SMILES Transformer例） ----
MODEL_NAME = "seyonec/ChemBERTa-zinc-base-v1"  # 公開モデル例（環境により変更OK）
BATCH_SIZE = 64
MAX_LEN    = 256  # 長すぎるSMILESは切られる（まずは256で十分なことが多い）

# ---- 学習器（回帰：CEそのものを予測する例） ----
from sklearn.linear_model import Ridge
from sklearn.model_selection import LeaveOneGroupOut
from scipy.stats import spearmanr

# ============================================================
# 1) CSV読込 & CEのクリーニング（NaNを除外）
# ============================================================
df = pd.read_csv(CSV_PATH)

# CEの文字列を数値化：
#  - "1.23%" みたいなケースに備えて % を除去
#  - 変換不能は NaN に落とす
ce_raw = df[COL_CE].astype(str).str.strip()
ce_raw = ce_raw.str.replace("%", "", regex=False)
# もし "1,234" みたいなカンマがあるなら除去
ce_raw = ce_raw.str.replace(",", "", regex=False)
df["CE_num"] = pd.to_numeric(ce_raw, errors="coerce")

# CEがNaNの行は除外（ユーザー要望）
df = df.dropna(subset=["CE_num"]).copy()
df = df.reset_index(drop=True)

print(f"[INFO] After dropping CE NaN: n = {len(df)}")

# ============================================================
# 2) SMILES妥当性チェック（RDKitでパースできない行を除外）
# ============================================================
from rdkit import Chem

def _fix_common_smiles_issues(s: str) -> str:
    """よくある軽微な崩れを最低限だけ直す（必要最小限）。
    - 前後空白除去
    - 末尾に余計な ']' が付くなど、明らかな過剰括弧を軽く修正
    ※危険な自動修正は避ける（化学的意味が変わる可能性があるため）
    """
    if s is None:
        return s
    s = str(s).strip()

    # 例: "FSI: [O=S(=O)([N-]S(=O)(=O)F)F]]" のような末尾余計 ']' を1つ落とす
    # RDKitが通らない場合のみ試すのが安全だが、ここでは軽微な修正のみ。
    if s.endswith("]]"):
        s_try = s[:-1]
        if Chem.MolFromSmiles(s_try) is not None:
            return s_try

    return s

def is_valid_smiles(s: str) -> bool:
    if s is None or (isinstance(s, float) and np.isnan(s)):
        return False
    s = str(s).strip()
    if s == "":
        return False
    try:
        return Chem.MolFromSmiles(s) is not None
    except Exception:
        return False

# 軽微修正→妥当性チェック
df[COL_CAT] = df[COL_CAT].map(_fix_common_smiles_issues)
df[COL_ANI] = df[COL_ANI].map(_fix_common_smiles_issues)

ok = df[COL_CAT].map(is_valid_smiles) & df[COL_ANI].map(is_valid_smiles)
bad_n = int((~ok).sum())
if bad_n > 0:
    print(f"[WARN] Dropping invalid SMILES rows: {bad_n}")
df = df[ok].copy().reset_index(drop=True)

print(f"[INFO] After dropping invalid SMILES: n = {len(df)}")

# ============================================================
# 3) 事前学習モデルから埋め込み抽出（カチオン/アニオン別にキャッシュ）
# ============================================================
import torch
from transformers import AutoTokenizer, AutoModel

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("[INFO] device =", device)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME).to(device)
model.eval()

@torch.no_grad()
def smiles_to_embedding(smiles_list, batch_size=BATCH_SIZE, max_len=MAX_LEN):
    """SMILES列 → 埋め込み行列 (N, H)。
    mean-pooling（attention_mask考慮）で分子全体のベクトルを作る。
    """
    all_emb = []
    for i in range(0, len(smiles_list), batch_size):
        batch = smiles_list[i:i+batch_size]
        tok = tokenizer(
            batch,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=max_len,
        )
        tok = {k: v.to(device) for k, v in tok.items()}
        out = model(**tok)  # last_hidden_state: (B, L, H)

        last = out.last_hidden_state
        mask = tok["attention_mask"].unsqueeze(-1)  # (B, L, 1)
        pooled = (last * mask).sum(dim=1) / mask.sum(dim=1)  # (B, H)
        all_emb.append(pooled.detach().cpu().numpy())

    return np.vstack(all_emb)

def embed_unique_smiles(smiles_series: pd.Series) -> np.ndarray:
    """ユニークSMILESを一度だけ埋め込み→元の順序に戻す"""
    smiles = smiles_series.astype(str).tolist()
    uniq = pd.unique(smiles_series.astype(str))
    uniq = [str(u) for u in uniq]

    print(f"[INFO] Embedding unique SMILES: {len(uniq)}")
    uniq_emb = smiles_to_embedding(uniq)

    emb_map = {s: e for s, e in zip(uniq, uniq_emb)}
    emb = np.vstack([emb_map[s] for s in smiles])
    return emb

# カチオン/アニオン別 embedding
E_cat = embed_unique_smiles(df[COL_CAT])
E_ani = embed_unique_smiles(df[COL_ANI])

# ============================================================
# 4) イオンペア特徴量化（結合）
# ============================================================
# ペア表現： [cat, ani, cat-ani, cat*ani]
X = np.hstack([E_cat, E_ani, E_cat - E_ani, E_cat * E_ani]).astype(np.float32)
y = df["CE_num"].to_numpy(dtype=np.float32)

print("[INFO] X shape:", X.shape, " y shape:", y.shape)

# ============================================================
# 5) 外挿評価（LeaveOneGroupOut）: cation-out / anion-out / ionpair-out
# ============================================================
def evaluate_logo_regression(X, y, groups, alpha=10.0):
    logo = LeaveOneGroupOut()
    pred = np.full_like(y, fill_value=np.nan, dtype=np.float32)

    # groupが1件しかないと分割が成立しないのでチェック
    uniq_groups = pd.Series(groups).astype(str).unique()
    if len(uniq_groups) < 2:
        raise ValueError("groupsの種類が1つしかありません。外挿分割できません。")

    for tr, te in logo.split(X, y, groups=groups):
        reg = Ridge(alpha=alpha)
        reg.fit(X[tr], y[tr])
        pred[te] = reg.predict(X[te])

    # Spearman（ランキング性能）
    rho = spearmanr(y, pred).correlation
    # RMSE（絶対値も一応）
    rmse = float(np.sqrt(np.mean((y - pred) ** 2)))
    return rho, rmse

# groups作成
groups_cat = df[COL_CAT].astype(str).to_numpy()
groups_ani = df[COL_ANI].astype(str).to_numpy()
groups_pair = (df[COL_CAT].astype(str) + "||" + df[COL_ANI].astype(str)).to_numpy()

print("\n===== Extrapolation: Leave-one-CATION-out =====")
rho, rmse = evaluate_logo_regression(X, y, groups_cat, alpha=10.0)
print(f"Spearman rho = {rho:.3f}, RMSE = {rmse:.4f}")

print("\n===== Extrapolation: Leave-one-ANION-out =====")
rho, rmse = evaluate_logo_regression(X, y, groups_ani, alpha=10.0)
print(f"Spearman rho = {rho:.3f}, RMSE = {rmse:.4f}")

print("\n===== Extrapolation: Leave-one-IONPAIR-out =====")
rho, rmse = evaluate_logo_regression(X, y, groups_pair, alpha=10.0)
print(f"Spearman rho = {rho:.3f}, RMSE = {rmse:.4f}")

# ============================================================
# 6) （任意）予測結果を保存
# ============================================================
# ここでは例として、cation-out の予測を保存したい場合のテンプレ
# （複数評価を保存したいなら evaluate_logo_regression を改造して pred も返すようにする）
print("\n[INFO] Done.")


[INFO] After dropping CE NaN: n = 63
[INFO] After dropping invalid SMILES: n = 63
[INFO] device = cuda
[INFO] Embedding unique SMILES: 17
[INFO] Embedding unique SMILES: 12
[INFO] X shape: (63, 3072)  y shape: (63,)

===== Extrapolation: Leave-one-CATION-out =====
Spearman rho = 0.568, RMSE = 2.9961

===== Extrapolation: Leave-one-ANION-out =====
Spearman rho = 0.263, RMSE = 4.5663

===== Extrapolation: Leave-one-IONPAIR-out =====
Spearman rho = 0.663, RMSE = 2.6019

[INFO] Done.


ChemBERTa: DeepChem/ChemBERTa-77M-MLM(Updated Jan 20, 2022)

In [ ]:
# ============================================
# SMILES(分子丸ごと) → 事前学習モデル埋め込み → 学習/外挿評価
# - CEがNaNの行はすべて除外
# - SMILESが壊れている行も除外（RDKitで検証）
# - カチオン/アニオンは別々に埋め込み→結合（おすすめ）
# - 「未知カチオン」「未知アニオン」「未知イオンペア」で外挿評価（LeaveOneGroupOut）
# ============================================

import os
import re
import numpy as np
import pandas as pd

# ---- 入力ファイル ----
CSV_PATH = "/content/cation_anion_26kept_withChelpG_fragments_DFT_extended_catprefix_15features_preprocessed.csv"

# ---- 列名（あなたのCSVに合わせる） ----
COL_CAT = "SMILES_cation"
COL_ANI = "SMILES_anion"
COL_CE  = "CE"          # CE（数値 or 文字列）を想定

# ---- 事前学習モデル（SMILES Transformer例） ----
MODEL_NAME = "DeepChem/ChemBERTa-77M-MLM"  # 公開モデル例（環境により変更OK）
BATCH_SIZE = 64
MAX_LEN    = 256  # 長すぎるSMILESは切られる（まずは256で十分なことが多い）

# ---- 学習器（回帰：CEそのものを予測する例） ----
from sklearn.linear_model import Ridge
from sklearn.model_selection import LeaveOneGroupOut
from scipy.stats import spearmanr

# ============================================================
# 1) CSV読込 & CEのクリーニング（NaNを除外）
# ============================================================
df = pd.read_csv(CSV_PATH)

# CEの文字列を数値化：
#  - "1.23%" みたいなケースに備えて % を除去
#  - 変換不能は NaN に落とす
ce_raw = df[COL_CE].astype(str).str.strip()
ce_raw = ce_raw.str.replace("%", "", regex=False)
# もし "1,234" みたいなカンマがあるなら除去
ce_raw = ce_raw.str.replace(",", "", regex=False)
df["CE_num"] = pd.to_numeric(ce_raw, errors="coerce")

# CEがNaNの行は除外（ユーザー要望）
df = df.dropna(subset=["CE_num"]).copy()
df = df.reset_index(drop=True)

print(f"[INFO] After dropping CE NaN: n = {len(df)}")

# ============================================================
# 2) SMILES妥当性チェック（RDKitでパースできない行を除外）
# ============================================================
from rdkit import Chem

def _fix_common_smiles_issues(s: str) -> str:
    """よくある軽微な崩れを最低限だけ直す（必要最小限）。
    - 前後空白除去
    - 末尾に余計な ']' が付くなど、明らかな過剰括弧を軽く修正
    ※危険な自動修正は避ける（化学的意味が変わる可能性があるため）
    """
    if s is None:
        return s
    s = str(s).strip()

    # 例: "FSI: [O=S(=O)([N-]S(=O)(=O)F)F]]" のような末尾余計 ']' を1つ落とす
    # RDKitが通らない場合のみ試すのが安全だが、ここでは軽微な修正のみ。
    if s.endswith("]]"):
        s_try = s[:-1]
        if Chem.MolFromSmiles(s_try) is not None:
            return s_try

    return s

def is_valid_smiles(s: str) -> bool:
    if s is None or (isinstance(s, float) and np.isnan(s)):
        return False
    s = str(s).strip()
    if s == "":
        return False
    try:
        return Chem.MolFromSmiles(s) is not None
    except Exception:
        return False

# 軽微修正→妥当性チェック
df[COL_CAT] = df[COL_CAT].map(_fix_common_smiles_issues)
df[COL_ANI] = df[COL_ANI].map(_fix_common_smiles_issues)

ok = df[COL_CAT].map(is_valid_smiles) & df[COL_ANI].map(is_valid_smiles)
bad_n = int((~ok).sum())
if bad_n > 0:
    print(f"[WARN] Dropping invalid SMILES rows: {bad_n}")
df = df[ok].copy().reset_index(drop=True)

print(f"[INFO] After dropping invalid SMILES: n = {len(df)}")

# ============================================================
# 3) 事前学習モデルから埋め込み抽出（カチオン/アニオン別にキャッシュ）
# ============================================================
import torch
from transformers import AutoTokenizer, AutoModel

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("[INFO] device =", device)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME).to(device)
model.eval()

@torch.no_grad()
def smiles_to_embedding(smiles_list, batch_size=BATCH_SIZE, max_len=MAX_LEN):
    """SMILES列 → 埋め込み行列 (N, H)。
    mean-pooling（attention_mask考慮）で分子全体のベクトルを作る。
    """
    all_emb = []
    for i in range(0, len(smiles_list), batch_size):
        batch = smiles_list[i:i+batch_size]
        tok = tokenizer(
            batch,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=max_len,
        )
        tok = {k: v.to(device) for k, v in tok.items()}
        out = model(**tok)  # last_hidden_state: (B, L, H)

        last = out.last_hidden_state
        mask = tok["attention_mask"].unsqueeze(-1)  # (B, L, 1)
        pooled = (last * mask).sum(dim=1) / mask.sum(dim=1)  # (B, H)
        all_emb.append(pooled.detach().cpu().numpy())

    return np.vstack(all_emb)

def embed_unique_smiles(smiles_series: pd.Series) -> np.ndarray:
    """ユニークSMILESを一度だけ埋め込み→元の順序に戻す"""
    smiles = smiles_series.astype(str).tolist()
    uniq = pd.unique(smiles_series.astype(str))
    uniq = [str(u) for u in uniq]

    print(f"[INFO] Embedding unique SMILES: {len(uniq)}")
    uniq_emb = smiles_to_embedding(uniq)

    emb_map = {s: e for s, e in zip(uniq, uniq_emb)}
    emb = np.vstack([emb_map[s] for s in smiles])
    return emb

# カチオン/アニオン別 embedding
E_cat = embed_unique_smiles(df[COL_CAT])
E_ani = embed_unique_smiles(df[COL_ANI])

# ============================================================
# 4) イオンペア特徴量化（結合）
# ============================================================
# ペア表現： [cat, ani, cat-ani, cat*ani]
X = np.hstack([E_cat, E_ani, E_cat - E_ani, E_cat * E_ani]).astype(np.float32)
y = df["CE_num"].to_numpy(dtype=np.float32)

print("[INFO] X shape:", X.shape, " y shape:", y.shape)

# ============================================================
# 5) 外挿評価（LeaveOneGroupOut）: cation-out / anion-out / ionpair-out
# ============================================================
def evaluate_logo_regression(X, y, groups, alpha=10.0):
    logo = LeaveOneGroupOut()
    pred = np.full_like(y, fill_value=np.nan, dtype=np.float32)

    # groupが1件しかないと分割が成立しないのでチェック
    uniq_groups = pd.Series(groups).astype(str).unique()
    if len(uniq_groups) < 2:
        raise ValueError("groupsの種類が1つしかありません。外挿分割できません。")

    for tr, te in logo.split(X, y, groups=groups):
        reg = Ridge(alpha=alpha)
        reg.fit(X[tr], y[tr])
        pred[te] = reg.predict(X[te])

    # Spearman（ランキング性能）
    rho = spearmanr(y, pred).correlation
    # RMSE（絶対値も一応）
    rmse = float(np.sqrt(np.mean((y - pred) ** 2)))
    return rho, rmse

# groups作成
groups_cat = df[COL_CAT].astype(str).to_numpy()
groups_ani = df[COL_ANI].astype(str).to_numpy()
groups_pair = (df[COL_CAT].astype(str) + "||" + df[COL_ANI].astype(str)).to_numpy()

print("\n===== Extrapolation: Leave-one-CATION-out =====")
rho, rmse = evaluate_logo_regression(X, y, groups_cat, alpha=10.0)
print(f"Spearman rho = {rho:.3f}, RMSE = {rmse:.4f}")

print("\n===== Extrapolation: Leave-one-ANION-out =====")
rho, rmse = evaluate_logo_regression(X, y, groups_ani, alpha=10.0)
print(f"Spearman rho = {rho:.3f}, RMSE = {rmse:.4f}")

print("\n===== Extrapolation: Leave-one-IONPAIR-out =====")
rho, rmse = evaluate_logo_regression(X, y, groups_pair, alpha=10.0)
print(f"Spearman rho = {rho:.3f}, RMSE = {rmse:.4f}")

# ============================================================
# 6) （任意）予測結果を保存
# ============================================================
# ここでは例として、cation-out の予測を保存したい場合のテンプレ
# （複数評価を保存したいなら evaluate_logo_regression を改造して pred も返すようにする）
print("\n[INFO] Done.")


[INFO] After dropping CE NaN: n = 63
[INFO] After dropping invalid SMILES: n = 63
[INFO] device = cuda


tokenizer_config.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/631 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/420 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/13.7M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at DeepChem/ChemBERTa-77M-MLM and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[INFO] Embedding unique SMILES: 17
[INFO] Embedding unique SMILES: 12
[INFO] X shape: (63, 1536)  y shape: (63,)

===== Extrapolation: Leave-one-CATION-out =====
Spearman rho = 0.536, RMSE = 3.1011

===== Extrapolation: Leave-one-ANION-out =====
Spearman rho = 0.235, RMSE = 3.7032

===== Extrapolation: Leave-one-IONPAIR-out =====
Spearman rho = 0.659, RMSE = 2.7399

[INFO] Done.


ChemBERTa: DeepChem/ChemBERTa-100M-MLM (Updated Aug 18, 2025)

In [ ]:
# ============================================
# SMILES(分子丸ごと) → 事前学習モデル埋め込み → 学習/外挿評価
# - CEがNaNの行はすべて除外
# - SMILESが壊れている行も除外（RDKitで検証）
# - カチオン/アニオンは別々に埋め込み→結合（おすすめ）
# - 「未知カチオン」「未知アニオン」「未知イオンペア」で外挿評価（LeaveOneGroupOut）
# ============================================

import os
import re
import numpy as np
import pandas as pd

# ---- 入力ファイル ----
CSV_PATH = "/content/cation_anion_26kept_withChelpG_fragments_DFT_extended_catprefix_15features_preprocessed.csv"

# ---- 列名（あなたのCSVに合わせる） ----
COL_CAT = "SMILES_cation"
COL_ANI = "SMILES_anion"
COL_CE  = "CE"          # CE（数値 or 文字列）を想定

# ---- 事前学習モデル（SMILES Transformer例） ----
MODEL_NAME = "DeepChem/ChemBERTa-100M-MLM"  # 公開モデル例（環境により変更OK）
BATCH_SIZE = 64
MAX_LEN    = 256  # 長すぎるSMILESは切られる（まずは256で十分なことが多い）

# ---- 学習器（回帰：CEそのものを予測する例） ----
from sklearn.linear_model import Ridge
from sklearn.model_selection import LeaveOneGroupOut
from scipy.stats import spearmanr

# ============================================================
# 1) CSV読込 & CEのクリーニング（NaNを除外）
# ============================================================
df = pd.read_csv(CSV_PATH)

# CEの文字列を数値化：
#  - "1.23%" みたいなケースに備えて % を除去
#  - 変換不能は NaN に落とす
ce_raw = df[COL_CE].astype(str).str.strip()
ce_raw = ce_raw.str.replace("%", "", regex=False)
# もし "1,234" みたいなカンマがあるなら除去
ce_raw = ce_raw.str.replace(",", "", regex=False)
df["CE_num"] = pd.to_numeric(ce_raw, errors="coerce")

# CEがNaNの行は除外（ユーザー要望）
df = df.dropna(subset=["CE_num"]).copy()
df = df.reset_index(drop=True)

print(f"[INFO] After dropping CE NaN: n = {len(df)}")

# ============================================================
# 2) SMILES妥当性チェック（RDKitでパースできない行を除外）
# ============================================================
from rdkit import Chem

def _fix_common_smiles_issues(s: str) -> str:
    """よくある軽微な崩れを最低限だけ直す（必要最小限）。
    - 前後空白除去
    - 末尾に余計な ']' が付くなど、明らかな過剰括弧を軽く修正
    ※危険な自動修正は避ける（化学的意味が変わる可能性があるため）
    """
    if s is None:
        return s
    s = str(s).strip()

    # 例: "FSI: [O=S(=O)([N-]S(=O)(=O)F)F]]" のような末尾余計 ']' を1つ落とす
    # RDKitが通らない場合のみ試すのが安全だが、ここでは軽微な修正のみ。
    if s.endswith("]]"):
        s_try = s[:-1]
        if Chem.MolFromSmiles(s_try) is not None:
            return s_try

    return s

def is_valid_smiles(s: str) -> bool:
    if s is None or (isinstance(s, float) and np.isnan(s)):
        return False
    s = str(s).strip()
    if s == "":
        return False
    try:
        return Chem.MolFromSmiles(s) is not None
    except Exception:
        return False

# 軽微修正→妥当性チェック
df[COL_CAT] = df[COL_CAT].map(_fix_common_smiles_issues)
df[COL_ANI] = df[COL_ANI].map(_fix_common_smiles_issues)

ok = df[COL_CAT].map(is_valid_smiles) & df[COL_ANI].map(is_valid_smiles)
bad_n = int((~ok).sum())
if bad_n > 0:
    print(f"[WARN] Dropping invalid SMILES rows: {bad_n}")
df = df[ok].copy().reset_index(drop=True)

print(f"[INFO] After dropping invalid SMILES: n = {len(df)}")

# ============================================================
# 3) 事前学習モデルから埋め込み抽出（カチオン/アニオン別にキャッシュ）
# ============================================================
import torch
from transformers import AutoTokenizer, AutoModel

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("[INFO] device =", device)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME).to(device)
model.eval()

@torch.no_grad()
def smiles_to_embedding(smiles_list, batch_size=BATCH_SIZE, max_len=MAX_LEN):
    """SMILES列 → 埋め込み行列 (N, H)。
    mean-pooling（attention_mask考慮）で分子全体のベクトルを作る。
    """
    all_emb = []
    for i in range(0, len(smiles_list), batch_size):
        batch = smiles_list[i:i+batch_size]
        tok = tokenizer(
            batch,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=max_len,
        )
        tok = {k: v.to(device) for k, v in tok.items()}
        out = model(**tok)  # last_hidden_state: (B, L, H)

        last = out.last_hidden_state
        mask = tok["attention_mask"].unsqueeze(-1)  # (B, L, 1)
        pooled = (last * mask).sum(dim=1) / mask.sum(dim=1)  # (B, H)
        all_emb.append(pooled.detach().cpu().numpy())

    return np.vstack(all_emb)

def embed_unique_smiles(smiles_series: pd.Series) -> np.ndarray:
    """ユニークSMILESを一度だけ埋め込み→元の順序に戻す"""
    smiles = smiles_series.astype(str).tolist()
    uniq = pd.unique(smiles_series.astype(str))
    uniq = [str(u) for u in uniq]

    print(f"[INFO] Embedding unique SMILES: {len(uniq)}")
    uniq_emb = smiles_to_embedding(uniq)

    emb_map = {s: e for s, e in zip(uniq, uniq_emb)}
    emb = np.vstack([emb_map[s] for s in smiles])
    return emb

# カチオン/アニオン別 embedding
E_cat = embed_unique_smiles(df[COL_CAT])
E_ani = embed_unique_smiles(df[COL_ANI])

# ============================================================
# 4) イオンペア特徴量化（結合）
# ============================================================
# ペア表現： [cat, ani, cat-ani, cat*ani]
X = np.hstack([E_cat, E_ani, E_cat - E_ani, E_cat * E_ani]).astype(np.float32)
y = df["CE_num"].to_numpy(dtype=np.float32)

print("[INFO] X shape:", X.shape, " y shape:", y.shape)

# ============================================================
# 5) 外挿評価（LeaveOneGroupOut）: cation-out / anion-out / ionpair-out
# ============================================================
def evaluate_logo_regression(X, y, groups, alpha=10.0):
    logo = LeaveOneGroupOut()
    pred = np.full_like(y, fill_value=np.nan, dtype=np.float32)

    # groupが1件しかないと分割が成立しないのでチェック
    uniq_groups = pd.Series(groups).astype(str).unique()
    if len(uniq_groups) < 2:
        raise ValueError("groupsの種類が1つしかありません。外挿分割できません。")

    for tr, te in logo.split(X, y, groups=groups):
        reg = Ridge(alpha=alpha)
        reg.fit(X[tr], y[tr])
        pred[te] = reg.predict(X[te])

    # Spearman（ランキング性能）
    rho = spearmanr(y, pred).correlation
    # RMSE（絶対値も一応）
    rmse = float(np.sqrt(np.mean((y - pred) ** 2)))
    return rho, rmse

# groups作成
groups_cat = df[COL_CAT].astype(str).to_numpy()
groups_ani = df[COL_ANI].astype(str).to_numpy()
groups_pair = (df[COL_CAT].astype(str) + "||" + df[COL_ANI].astype(str)).to_numpy()

print("\n===== Extrapolation: Leave-one-CATION-out =====")
rho, rmse = evaluate_logo_regression(X, y, groups_cat, alpha=10.0)
print(f"Spearman rho = {rho:.3f}, RMSE = {rmse:.4f}")

print("\n===== Extrapolation: Leave-one-ANION-out =====")
rho, rmse = evaluate_logo_regression(X, y, groups_ani, alpha=10.0)
print(f"Spearman rho = {rho:.3f}, RMSE = {rmse:.4f}")

print("\n===== Extrapolation: Leave-one-IONPAIR-out =====")
rho, rmse = evaluate_logo_regression(X, y, groups_pair, alpha=10.0)
print(f"Spearman rho = {rho:.3f}, RMSE = {rmse:.4f}")

# ============================================================
# 6) （任意）予測結果を保存
# ============================================================
# ここでは例として、cation-out の予測を保存したい場合のテンプレ
# （複数評価を保存したいなら evaluate_logo_regression を改造して pred も返すようにする）
print("\n[INFO] Done.")


[INFO] After dropping CE NaN: n = 63
[INFO] After dropping invalid SMILES: n = 63
[INFO] device = cuda


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at DeepChem/ChemBERTa-100M-MLM and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[INFO] Embedding unique SMILES: 17
[INFO] Embedding unique SMILES: 12
[INFO] X shape: (63, 3072)  y shape: (63,)

===== Extrapolation: Leave-one-CATION-out =====
Spearman rho = 0.531, RMSE = 3.0492

===== Extrapolation: Leave-one-ANION-out =====
Spearman rho = 0.502, RMSE = 3.4737

===== Extrapolation: Leave-one-IONPAIR-out =====
Spearman rho = 0.660, RMSE = 2.6934

[INFO] Done.


(1) anion-out をアニオン別に分解し、(2) PCA＋Ridge(α探索)を“外側分割の中で”リーク無しで実行し、さらに **(3) ペア埋め込み（cat.ani を1回で埋め込み）**も同じ枠組みで比較できる「一式コード」

In [ ]:
# ============================================
# 1) anion-out をアニオン別に分解
# 2) PCA + Ridge(alpha探索) をリーク無しで評価
# 3) separate埋め込み(現行: cat/ani/diff/prod) と pair埋め込み(cat.ani) を比較
# ============================================

import numpy as np
import pandas as pd

CSV_PATH = "/content/cation_anion_26kept_withChelpG_fragments_DFT_extended_catprefix_15features_preprocessed.csv"
COL_CAT, COL_ANI, COL_CE = "SMILES_cation", "SMILES_anion", "CE"

# ---- 評価候補（必要に応じて変更） ----
ALPHAS = [0.1, 1, 10, 100, 1000]
PCA_DIMS = [None, 64, 128, 256]   # None = PCAなし
MAX_LEN = 256
BATCH_SIZE = 64

# ---- 使う事前学習モデル（あなたが比較した3つ）----
MODEL_LIST = [
    "seyonec/ChemBERTa-zinc-base-v1",
    "DeepChem/ChemBERTa-77M-MLM",
    "DeepChem/ChemBERTa-100M-MLM",
]

# ============================================
# 0) データ前処理（CE NaN除外、SMILES妥当性チェック）
# ============================================
from rdkit import Chem

def to_numeric_ce(s: pd.Series) -> pd.Series:
    x = s.astype(str).str.strip()
    x = x.str.replace("%", "", regex=False)
    x = x.str.replace(",", "", regex=False)
    return pd.to_numeric(x, errors="coerce")

def is_valid_smiles(smiles: str) -> bool:
    if smiles is None:
        return False
    smiles = str(smiles).strip()
    if smiles == "":
        return False
    try:
        return Chem.MolFromSmiles(smiles) is not None
    except Exception:
        return False

df = pd.read_csv(CSV_PATH)
df["CE_num"] = to_numeric_ce(df[COL_CE])

df = df.dropna(subset=["CE_num"]).copy()
df = df[df[COL_CAT].map(is_valid_smiles) & df[COL_ANI].map(is_valid_smiles)].copy()
df = df.reset_index(drop=True)

y = df["CE_num"].to_numpy(dtype=float)
print("[INFO] usable rows:", len(df))

# ============================================
# 1) 埋め込み抽出（ユニークSMILESをキャッシュ）
# ============================================
import torch
from transformers import AutoTokenizer, AutoModel

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("[INFO] device:", device)

@torch.no_grad()
def smiles_to_embedding(smiles_list, tokenizer, model, batch_size=BATCH_SIZE, max_len=MAX_LEN):
    embs = []
    for i in range(0, len(smiles_list), batch_size):
        batch = smiles_list[i:i+batch_size]
        tok = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=max_len)
        tok = {k: v.to(device) for k, v in tok.items()}
        out = model(**tok)
        last = out.last_hidden_state
        mask = tok["attention_mask"].unsqueeze(-1)
        pooled = (last * mask).sum(dim=1) / mask.sum(dim=1)
        embs.append(pooled.detach().cpu().numpy())
    return np.vstack(embs)

def embed_series_unique(series: pd.Series, tokenizer, model) -> np.ndarray:
    # seriesの順序に合わせて埋め込みを返す（ユニークだけ計算）
    arr = series.astype(str).to_numpy()
    uniq = pd.unique(arr)
    uniq = [str(u) for u in uniq]
    print(f"[INFO] Embedding unique strings: {len(uniq)}")
    uniq_emb = smiles_to_embedding(uniq, tokenizer, model)
    m = {s:e for s,e in zip(uniq, uniq_emb)}
    return np.vstack([m[s] for s in arr])

# ============================================
# 2) 特徴量構成（separate / pair）
# ============================================
def build_features(df: pd.DataFrame, tokenizer, model, mode: str) -> np.ndarray:
    """
    mode:
      - "separate_4x": E_cat, E_ani, (E_cat-E_ani), (E_cat*E_ani) → 3072次元
      - "pair_only": pair_smiles = cat + "." + ani を1回でembedding → 768次元
    """
    if mode == "separate_4x":
        E_cat = embed_series_unique(df[COL_CAT], tokenizer, model)
        E_ani = embed_series_unique(df[COL_ANI], tokenizer, model)
        X = np.hstack([E_cat, E_ani, E_cat - E_ani, E_cat * E_ani]).astype(np.float32)
        return X

    if mode == "pair_only":
        pair = (df[COL_CAT].astype(str) + "." + df[COL_ANI].astype(str))
        E_pair = embed_series_unique(pair, tokenizer, model)
        return E_pair.astype(np.float32)

    raise ValueError("Unknown mode")

# ============================================
# 3) リーク無し：外側LOGOの中でPCA/alphaを選ぶ
# ============================================
from sklearn.model_selection import LeaveOneGroupOut, GroupKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from scipy.stats import spearmanr

def inner_select_model(X_tr, y_tr, groups_tr, alphas=ALPHAS, pca_dims=PCA_DIMS, random_state=0):
    """
    外側foldの学習データのみを使って、(PCA次元, alpha) を選ぶ。
    inner CV も group を保つ（GroupKFold）。
    """
    # group種類が少なすぎるとGroupKFoldできないので調整
    uniq = pd.unique(groups_tr)
    n_groups = len(uniq)
    n_splits = min(5, n_groups)
    if n_splits < 2:
        # どうにもならない場合は固定で返す
        return (None, 10.0)

    gkf = GroupKFold(n_splits=n_splits)

    best = None
    best_score = -np.inf

    for d in pca_dims:
        for a in alphas:
            scores = []
            for itr, ite in gkf.split(X_tr, y_tr, groups=groups_tr):
                X_i_tr, y_i_tr = X_tr[itr], y_tr[itr]
                X_i_te, y_i_te = X_tr[ite], y_tr[ite]

                steps = [
                    ("scaler", StandardScaler(with_mean=True, with_std=True)),
                ]
                if d is not None:
                    steps.append(("pca", PCA(n_components=d, svd_solver="randomized", random_state=random_state)))
                steps.append(("ridge", Ridge(alpha=a)))

                pipe = Pipeline(steps)
                pipe.fit(X_i_tr, y_i_tr)
                pred = pipe.predict(X_i_te)

                # 目的がランキング寄りなのでSpearmanで選ぶ（RMSEで選びたいなら差し替え）
                rho = spearmanr(y_i_te, pred).correlation
                if np.isnan(rho):
                    rho = -1.0
                scores.append(rho)

            mean_rho = float(np.mean(scores))
            if mean_rho > best_score:
                best_score = mean_rho
                best = (d, a)

    return best

def outer_logo_eval(X, y, groups, outer_name="group"):
    """
    外側：LeaveOneGroupOut
    内側：学習データ内で(PCA次元, alpha)選択（リーク無し）
    """
    logo = LeaveOneGroupOut()

    pred_all = np.zeros_like(y, dtype=float)
    fold_rows = []

    for fold_id, (tr, te) in enumerate(logo.split(X, y, groups=groups), start=1):
        X_tr, y_tr = X[tr], y[tr]
        X_te, y_te = X[te], y[te]
        g_tr = groups[tr]
        g_te_val = groups[te][0]  # foldは同一group

        best_d, best_a = inner_select_model(X_tr, y_tr, g_tr)

        steps = [("scaler", StandardScaler(with_mean=True, with_std=True))]
        if best_d is not None:
            steps.append(("pca", PCA(n_components=best_d, svd_solver="randomized", random_state=0)))
        steps.append(("ridge", Ridge(alpha=best_a)))
        pipe = Pipeline(steps)

        pipe.fit(X_tr, y_tr)
        pred = pipe.predict(X_te)
        pred_all[te] = pred

        rmse = float(np.sqrt(mean_squared_error(y_te, pred)))
        rho = spearmanr(y_te, pred).correlation
        # テストが1点しかないfoldはSpearmanが定義できない
        if np.isnan(rho):
            rho = np.nan

        fold_rows.append({
            "outer": outer_name,
            "fold": fold_id,
            "group": g_te_val,
            "n_test": len(te),
            "best_pca_dim": best_d,
            "best_alpha": best_a,
            "fold_rmse": rmse,
            "fold_spearman": rho,
        })

    overall_rho = spearmanr(y, pred_all).correlation
    overall_rmse = float(np.sqrt(mean_squared_error(y, pred_all)))

    return pred_all, pd.DataFrame(fold_rows), overall_rho, overall_rmse

# ============================================
# 4) まとめて実行：各モデル×(separate / pair)で3種類外挿
# ============================================
ALL_SUMMARY = []
ALL_FOLDS = []

feature_modes = ["separate_4x", "pair_only"]

for model_name in MODEL_LIST:
    print("\n" + "="*80)
    print("[MODEL]", model_name)

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name).to(device)
    model.eval()

    for mode in feature_modes:
        print("\n--- feature mode:", mode, "---")
        X = build_features(df, tokenizer, model, mode=mode)
        print("[INFO] X shape:", X.shape)

        # groups
        groups_cat = df[COL_CAT].astype(str).to_numpy()
        groups_ani = df[COL_ANI].astype(str).to_numpy()
        groups_pair = (df[COL_CAT].astype(str) + "||" + df[COL_ANI].astype(str)).to_numpy()

        # 1) cation-out
        _, folds, rho, rmse = outer_logo_eval(X, y, groups_cat, outer_name="cation_out")
        folds["model"] = model_name
        folds["feat_mode"] = mode
        ALL_FOLDS.append(folds)
        ALL_SUMMARY.append({"model": model_name, "feat_mode": mode, "outer": "cation_out", "rho": rho, "rmse": rmse})

        # 2) anion-out（アニオン別に分解する対象）
        _, folds, rho, rmse = outer_logo_eval(X, y, groups_ani, outer_name="anion_out")
        folds["model"] = model_name
        folds["feat_mode"] = mode
        ALL_FOLDS.append(folds)
        ALL_SUMMARY.append({"model": model_name, "feat_mode": mode, "outer": "anion_out", "rho": rho, "rmse": rmse})

        # 3) ionpair-out
        _, folds, rho, rmse = outer_logo_eval(X, y, groups_pair, outer_name="ionpair_out")
        folds["model"] = model_name
        folds["feat_mode"] = mode
        ALL_FOLDS.append(folds)
        ALL_SUMMARY.append({"model": model_name, "feat_mode": mode, "outer": "ionpair_out", "rho": rho, "rmse": rmse})

# 結果テーブル
summary_df = pd.DataFrame(ALL_SUMMARY).sort_values(["outer","feat_mode","model"])
folds_df = pd.concat(ALL_FOLDS, ignore_index=True)

print("\n" + "#"*80)
print("SUMMARY (overall):")
print(summary_df)

print("\n" + "#"*80)
print("ANION-OUT folds (per anion):")
print(
    folds_df[folds_df["outer"]=="anion_out"]
    .sort_values(["feat_mode","model","fold_rmse"], ascending=[True, True, False])
    [["model","feat_mode","group","n_test","best_pca_dim","best_alpha","fold_rmse","fold_spearman"]]
)

# 解析用に保存（任意）
summary_df.to_csv("/mnt/data/chemberta_transfer_summary.csv", index=False)
folds_df.to_csv("/mnt/data/chemberta_transfer_folds.csv", index=False)
print("\n[INFO] saved: /mnt/data/chemberta_transfer_summary.csv")
print("[INFO] saved: /mnt/data/chemberta_transfer_folds.csv")


[INFO] usable rows: 63
[INFO] device: cuda

[MODEL] seyonec/ChemBERTa-zinc-base-v1

--- feature mode: separate_4x ---
[INFO] Embedding unique strings: 17
[INFO] Embedding unique strings: 12
[INFO] X shape: (63, 3072)


ValueError: n_components=64 must be between 1 and min(n_samples, n_features)=47 with svd_solver='randomized'